<a href="https://colab.research.google.com/github/CamiloVga/Codes/blob/main/Git_AppPeriodistaAI_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# App todo en uno de producción de noticias

Código paso a paso para crear una app de gradio para escribir noticia por medio de GPT-3.5-turbo y Whisper de OpenAi

La herramienta recibe y transcribe audios de reportería, entiende los hechos y las instrucciones para escribir un contenido funcional.

Creado por: [Camilo Vega ](https://www.linkedin.com/in/camilo-vega-169084b1/)

In [ ]:
# Verificar que las librerías estén instaladas correctamente
!pip install openai==0.28
!pip install transformers
!pip install git+https://github.com/openai/whisper.git
!pip install gradio

# Importar librerías
import openai
import whisper
import tempfile
import gradio as gr
import os
from gradio import processing_utils as gr_proc

# Configurar la clave API de OpenAI de manera más segura
openai.api_key = "API KEY"

# Descargar modelo Whisper una vez
model = whisper.load_model("base")

# Funciones
def periodista_todo_en_uno(instrucciones, texto_info, archivos_audio):
    try:
        transcripciones = []
        for archivo in archivos_audio:
            try:
                if isinstance(archivo, str):
                    archivo_path = archivo
                else:
                    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_file:
                        temp_file.write(archivo.read())
                        temp_file.flush()
                        archivo_path = temp_file.name

                resultado = model.transcribe(archivo_path)
                transcripciones.append(resultado['text'])

                # Eliminar archivo temporal
                os.unlink(archivo_path)

            except Exception as e:
                return f"Error al procesar el archivo de audio: {str(e)}", "", ""

        prompt = f"{instrucciones}\n\nInformación adicional: {texto_info}\n\nTranscripciones de audio: {' '.join(transcripciones)}"

        try:
            respuesta = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=[{"role": "user", "content": prompt}]
            )
            borrador = respuesta.choices[0].message.get("content", "")
        except Exception as e:
            return f"Error al generar el contenido: {str(e)}", "", ""

        titulo = generar_titulo(instrucciones)
        sumario = generar_sumario(borrador)
        cuerpo = borrador
        return titulo, sumario, cuerpo

    except Exception as e:
        return f"Error: {str(e)}", "", ""

def generar_titulo(instrucciones):
    try:
        prompt = f"Genera un título de 5-10 palabras para un artículo sobre: {instrucciones}"
        respuesta = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[{"role": "user", "content": prompt}]
        )
        return respuesta.choices[0].message.get("content", "")
    except Exception as e:
        return f"Error al generar título: {str(e)}"

def generar_sumario(borrador):
    try:
        prompt = f"Crea un sumario de 20 palabras para el siguiente artículo: {borrador}"
        respuesta = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[{"role": "user", "content": prompt}]
        )
        return respuesta.choices[0].message.get("content", "")
    except Exception as e:
        return f"Error al generar sumario: {str(e)}"

# Interfaz de Gradio
inputs = [
  gr.Textbox(label="Instrucciones",
              value="Escribe una noticia al estilo New York times sobre.."),
  gr.Textbox(label="Describes los hechos de la noticia"),
  gr.File(label="Archivos de audio", file_count="multiple", type="filepath")

]
outputs = [
  gr.Textbox(label="Título"),
  gr.Textbox(label="Sumario"),
  gr.Textbox(label="Cuerpo del contenido")
]
title = "Generador de noticias todo en uno"
description = "Esta aplicación es un servicio todo en uno para escribir una noticia: indica los hechos, sube los audios de la reportería y con las instrucciones elige el tono y estilo del contenido"

gr.Interface(
  fn=periodista_todo_en_uno,
  inputs=inputs,
  outputs=outputs,
  title=title,
  description=description
).launch()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-xddog3et
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-xddog3et
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manyli

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 122MiB/s]


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9143cdda86241a23ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
